In [1]:
# import dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# get data from database
Base = automap_base()
engine = create_engine('sqlite:///../data/fbdata.sqlite')
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)
modelRecent = pd.read_sql_query("SELECT * FROM RecentQBs", con=engine)

In [3]:
# load in model
from tensorflow.keras.models import load_model
model = load_model("fullModel.h5")

from joblib import dump, load
X_scaler=load('X_scaler.bin')

In [4]:
# test with recent data
X_recent = modelRecent[['height','40yard','vertleap','rushAvg', 'adjYdsPerAtt','effRate']]
X_recent_scaled = X_scaler.transform(X_recent)
y_recent = modelRecent['predictedSuccess']
y_recent_categorical = to_categorical(y_recent)


testOutcomes = modelRecent[modelRecent['DraftYear'] < 2020][['Player','predictedSuccess']]
test_loss, test_accuracy, test_precision = model.evaluate(
    X_recent_scaled, y_recent_categorical, verbose=2)
print(f"Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {test_precision}")
print("-------------------------------------------------------")


predictions = model.predict_classes(X_recent_scaled)
for i in range(0,len(testOutcomes)):
    print(f"Prediction for {testOutcomes['Player'][i]}: {predictions[i]}")
    print(f"Actual for {testOutcomes['Player'][i]}: {int(testOutcomes['predictedSuccess'][i])}")
    print("-------------------------------------------------------")

65/65 - 0s - loss: 3.9942 - accuracy: 0.5692 - Precision: 0.5692
Loss: 3.994202276949699, Accuracy: 0.5692307949066162, Precision: 0.5692307949066162
-------------------------------------------------------
Prediction for Jameis Winston: 1
Actual for Jameis Winston: 1
-------------------------------------------------------
Prediction for Marcus Mariota: 1
Actual for Marcus Mariota: 0
-------------------------------------------------------
Prediction for Garrett Grayson: 1
Actual for Garrett Grayson: 0
-------------------------------------------------------
Prediction for Sean Mannion: 0
Actual for Sean Mannion: 0
-------------------------------------------------------
Prediction for Bryce Petty: 0
Actual for Bryce Petty: 0
-------------------------------------------------------
Prediction for Brett Hundley: 1
Actual for Brett Hundley: 0
-------------------------------------------------------
Prediction for Trevor Siemian: 0
Actual for Trevor Siemian: 0
----------------------------------

In [5]:
# test with recent data
X_2020 = modelRecent[modelRecent['DraftYear'] >= 2019][['height','40yard','vertleap','rushAvg', 'adjYdsPerAtt','effRate']]
X_2020_scaled = X_scaler.transform(X_2020)
players_2020 = modelRecent[modelRecent['DraftYear'] >= 2019][['Player','DraftYear']].reset_index()
del players_2020['index']

predictions = model.predict_classes(X_recent_scaled)
for i in range(0,len(players_2020)):
    print(f"Prediction for {players_2020['Player'][i]} ({players_2020['DraftYear'][i]}): {predictions[i]}")
    print("-------------------------------------------------------")

Prediction for Kyler Murray (2019): 1
-------------------------------------------------------
Prediction for Daniel Jones (2019): 1
-------------------------------------------------------
Prediction for Dwayne Haskins (2019): 1
-------------------------------------------------------
Prediction for Drew Lock (2019): 0
-------------------------------------------------------
Prediction for Will Grier (2019): 0
-------------------------------------------------------
Prediction for Ryan Finley (2019): 1
-------------------------------------------------------
Prediction for Jarrett Stidham (2019): 0
-------------------------------------------------------
Prediction for Clayton Thorson (2019): 0
-------------------------------------------------------
Prediction for Gardner Minshew (2019): 1
-------------------------------------------------------
Prediction for Trace McSorley (2019): 1
-------------------------------------------------------
Prediction for Joe Burrow (2020): 1
-----------------